# For Learning ML with Tensorflow

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
from tqdm.notebook import tqdm as tq
import numpy as np
import matplotlib.pyplot as plt

TensorFlow 1.x selected.


## 1. Linear Regression and Classification
Cost function, Gradient descent, Mini batch

### 1-1. Linear Regression


Cost function -> MSE <br>
How to minimize **cost**?  -> Using Gradient Descent 

In [0]:
# x_data = np.array([[1],[2],[3]], dtype=np.float32)
# y_data = np.array([[1],[2],[3]], dtype=np.float32)
x_data = [1,2,3]
y_data = [1,2,3]

X = tf.placeholder(tf.float32, name='X_input')
Y = tf.placeholder(tf.float32, name='Y_input')

W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

#Model
hypothesis = X*W + b 

cost = tf.reduce_mean(tf.square(hypothesis-Y)) #Cost function
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

## If you want to get gradients
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
# gvs = optimizer.compute_gradients(cost)
# apply_gradients = optimizer.apply_gradients(gvs)
# sess.run(apply_gradients)

with tf.Session() as sess: ## Tensorflow 1에서는 Session을 이용해 계산해 주어야 한다.
  sess.run(tf.global_variables_initializer()) ##Initializes global variables in the graph.

  for epoch in tq(range(2001)):
    sess.run(optimizer, feed_dict={X:x_data,Y:y_data})

    if epoch%100 == 0:
      print(f'{epoch} W : {sess.run(W)}, b : {sess.run(b)} loss : {sess.run(cost,feed_dict={X:x_data,Y:y_data})}')

line_x = np.arange(min(x_data),max(x_data),0.01)
line_y = W*line_x + b
plt.plot(line_x,line_y,'-r')
plt.plot(X,Y,'bo')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

In [0]:
# Tensorflow 2
# X = [1,2,3]
# Y = [1,2,3]

# # Model
# W = tf.Variable(tf.random.normal([1]), name='weight')
# b = tf.Variable(tf.random.normal([1]), name='bias')

# def compute_cost():
#   y_pred = W * X + b
#   cost = tf.reduce_mean(tf.square(Y-y_pred))
#   return cost

# optimizer = tf.optimizers.SGD(learning_rate = 0.01)

# for i in tq(range(1000)):
#   optimizer.minimize(compute_cost, [W,b])

#   if i%100 == 0:
#     print(f'{i} W : {W.numpy()}, b : {b.numpy()} loss : {compute_cost().numpy()}')

# line_x = np.arange(min(X),max(X),0.01)
# line_y = W*line_x + b

# plt.plot(line_x,line_y,'-r')
# plt.plot(X,Y,'bo')
# plt.xlabel('X')
# plt.ylabel('Y')
# plt.show()

###1-2. Multi Varialbe Linear Regression
with mini-batch

Lecture(theory) : H(x) = Wx + b <br>
In TensorFlow : H(X) = XW -> Because of matrix

In [0]:
x_data = [[73., 80., 75.],
          [93., 88., 93.],
          [89., 91., 90.],
          [96., 98., 100.],
          [73., 66., 70.]]
y_data = [[152.],
          [185.],
          [180.],
          [196.],
          [142.]]

X = tf.placeholder(tf.float32, shape=[None,3], name='X_input')
Y = tf.placeholder(tf.float32, shape=[None,1], name='Y_input')

W = tf.Variable(tf.random.normal([3,1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

hypothesis = tf.matmul(X, W) + b # tf.matmaul => 행렬곱

cost = tf.reduce_mean(tf.square(hypothesis - Y))
train_step = tf.train.GradientDescentOptimizer(learning_rate=0.00001).minimize(cost)

with tf.Session() as sess:
 sess.run(tf.global_variables_initializer())

 for epoch in tq(range(2001)):
   cost_val, hy_val, _ = sess.run([cost, hypothesis, train_step], feed_dict={X:x_data,Y:y_data})
   
   if epoch % 100 == 0:
     print(f'{epoch}  | Cost: {cost_val}\nPrediction:\n{hy_val}')

In [0]:
# Data load using tf reader

filenames = tf.train.string_input_producer(['xxxxx.csv'], 
                                                shuffle=False, name='filenames')

key, value = tf.TextLineReader().read(filenames)

record_defaults = [[0.], [0.], [0.], [0.]]
data = tf.decode_csv(value, record_defaults=record_defaults)

train_x_batch, train_y_batch = tf.train.batch([data[:-1], data[-1:]], batch_size=10)

X = tf.placeholder(tf.float32, shape=[None,3], name='X_input')
Y = tf.placeholder(tf.float32, shape=[None,1], name='Y_input')

W = tf.Variable(tf.random.normal([3,1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

hypothesis = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.square(hypothesis - Y))
train_step = tf.train.GradientDescentOptimizer(learning_rate=1e-5).minimize(cost)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  # Start population the filenames, Batch를 관리하기 위한 것 같음. 통상적으로 이렇게 사용
  coord = tf.train.Coordinator()
  threads = tf.train.start_queue_runners(sess=sess, coord = coord)

  for epoch in range(2001):
      x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
      cost_val, hy_val, _ = sess.run(
          [cost, hypothesis, train_step], feed_dict={X: x_batch, Y: y_batch})
      if epoch % 100 == 0:
        print(f'{epoch}  | Cost: {cost_val}\nPrediction:\n{hy_val}')

  coord.request_stop()
  coord.join(threads)


## Estimate the value through the model created.
print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

###1-3. Logistic Classification
predict 0 or 1

Using simoid function.

In [0]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data = [[0],[0],[0],[1],[1],[1]]

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 2])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([2, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# 값을 0~1 사이로 나오게 하기 위해 sigmoid 함수를 취해줌
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train_step = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

#cast Y_pred to 0 or 1
y_pred = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_pred, Y), dtype=tf.float32))

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for eopch in range(10001):
        cost_val, _ = sess.run([cost, train_step], feed_dict={X: x_data, Y: y_data})
        if epoch % 200 == 0:
            print(epoch, cost_val)

    # Accuracy report
    h, c, a = sess.run([hypothesis, y_pred, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print(f"\nHypothesis: '{h}'\nCorrect (Y): '{c}'\nAccuracy: '{a}' ")


##2. Softmax Regression
predict 0 to 1

softmax has predict value as probability.
The sum of all probabilities is 1. <br>
Cost function : Cross - Entropy.
Optimizer : GD

Softmax는 이진화된 값(Binary Value)으로 표현하기 위해 One-Hot Encoding이라는 것을 해주어야 한다. <br>
단순한 Integer Encoding의 범주형 값(Categorical Value)은 잘못된 경향성을 학습할 수 있다. EX) dog(1), cat(2), horse(3)이 있을 때 dog와 horse의 평균을 cat이라 생각할 수 있다.

In [0]:
# Load data
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

nb_classes = 7

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])

# In TF, if the input indices is rank N, the output will have rank N+1.
# So you have to reshape the output.
Y_one_hot = tf.one_hot(Y, nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

logits = tf.matmul(X, W) + b
y_pred = tf.nn.softmax(logits)

# tf.nn.softmax_cross_entropy_with_logits API -> One-hot Encoding 형태의 정답 레이블일 때 사용.
# tf.nn.sparse_softmax_corss_entropy_with_logits API -> One-hot Encoding을 자동으로 수행함으로 labels에 범주형 값 대입.
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=tf.stop_gradient([Y_one_hot])))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# tf.argmax는 probability중 가장 높은 probabliity를 스칼라 형태로 리턴.
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(Y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(2001):
        _, cost_val, acc_val = sess.run([optimizer, cost, accuracy], feed_dict={X: x_data, Y: y_data})
                                        
        if epoch % 100 == 0:
            print("Step: {:5}\tCost: {:.3f}\tAcc: {:.2%}".format(step, cost_val, acc_val))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print(f"[{p==int(y)}] Prediction: {p} True Y: {int(y)}")
        

##3. Learning rate, Preprocessing(Normalization), Overfitting(Regularization), Dataset


#### Learning rate
* Large learning rate - Cost가 계속 증가한다.
* Small learning rate - Cost 변화가 너무 없다.
` 결국 Learning rate은 Cost를 관찰하여 직접 설정해햐 한다.`
-----
#### Preprocessing(Normalization)
* zero-centered data : value의 중심을 0으로 바꿔주는 것
* normalized data : value값의 범위를 정해주는 것, 너무 튀는 값은 train data에서 제거해준다.
  > Standardization(normalization의 한 종류)
  > ```python
  > x_std[:,0] = (x[:,0] - x[:,0].mean()) / x[:,0].std()
  > ```
  > std() = 표준편차
------
#### Overfitting
Regularization은 Overfitting을 줄이기 위해 무거운 weight을 줄여주는 방식. cost함수 뒤에 텀을 추가. Regularization strength를 설정.
```python
l2reg = Regularization_strength * tf.reduce_sum(tf.square(W))
```
-----
#### Training Data, Validation Data, Test data
전체 데이터를 `트레이닝`, `검증`, `테스트`의 총 3가지로 데이터를 분류하는 것. 검증용 데이터는 트레이닝 과정에서 학습에 사용하지는 않지만 중간중간 테스트하는데 사용해서 학습하고있는 모델이 오버피팅에 빠지지 않는지 체크한다. 
그리고 마지막으로 테스트 데이터를 통해 모델의 정확도를 확인한다.
